# I See Dead People: An Analysis

Bryan Bumgardner, Data Scientist  
February - March 2016

### Death is a fact of life.   
That being said, it's worth studying how it fits in our popular culture. During the Metis Data Science bootcamp (shameless plug), while working on another project I discovered bodycounters.com[http://www.bodycounters.com/]. The dedicated volunteers at this site count and categorize the number of deaths in movies, and to date, have counted for over 2,000 movies. Go grab some beers, count the dead people in your favorite movie, and contribute to the site. 

The data are sitting on their site for anyone to see, and it gave me an idea. I reached out to the site and had some correspondence with Dana, who was gracious enough to share a CSV of all the data. I then cross referenced this data with information from another site, thenumbers.com, which shares basic budget and financial information about thousands of movies. I took this data, mashed it together, and asked some questions. 

Thanks again to the volunteers at bodycounters. Y'all are the greatest. Below is a quick data science passion project to analyze and find patterns in this great data you've collected. 

### Major questions:
1. Is the amount of on-screen deaths going up in modern movies?
2. Does a movie's death count have an effect on that movie's domestic gross?
3. Does a movie's death count have an effect on that movie's rating? 
4. How does budget factor into a movie's death count?
5. Is there a relationship between the movie's MPAA rating and body count?


### Table of Contents:
1. Reading and combining the data
2. Outputting various data frames
3. Visualizing
4. Conclusions
5. Documentation of the data

In [1]:
import pandas as pd 
import numpy as np
import csv
import os

import codecs
from bs4 import BeautifulSoup

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression 
from patsy import dmatrices


import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

%matplotlib inline
sns.set(color_codes=True)

### Step 1: read in and combine the data

Deaths data is a csv...

In [2]:
deaths = pd.read_csv("moviesmovies.csv", encoding="latin-1", index_col="movieskey") #might as well make use of their key

In [3]:
deaths["moviename"] = deaths["moviename"].str.rstrip("\t").str.lower() #cleaning some trailing spaces and standardizing for the matching
deaths["deathcomment"] = deaths["deathcomment"].str.rstrip("\t")

In [4]:
deaths.head(6)

,moviename,deathcount,deathcomment
movieskey,,,
1,king kong,71,"26 with Ape. Also 3 dinosaurs, 4 T-Rexes, 35 v..."
2,wrong turn,8,8
3,death to the supermodels,7,7
4,zombie honeymoon,10,10
5,ginger dead man,7,6 and a cookie
6,punisher,58,58


Grabbed a lot of the budget data from the website by saving it as a html, because the site crashes nonstop. Fixing:

In [5]:
budgetwebsite = pd.read_html("allmoviebudgets.html", header=0, encoding="utf-8")

In [6]:
all_budgets = pd.DataFrame(budgetwebsite[0])

How do I link them together? Fuzzy matching won't work: think "Terminator 2" and "Terminator 3." It would work if I had more data from the deaths source. I know how to do this in Excel... 

In [7]:
all_budgets.to_csv("all_budgets.csv", encoding='utf-8')

Ok so I'm really lazy and that only matched like maybe a third of the movies. Let's scrape all the movie data - EVER - because we are crazy. And also because I'm not interested in paying for access to their database. 

So what you see below is me scraping all the budgets and domestic box offices for every movie in the alphabetized database. Every letter has a chart with all the movies starting with that letter. I downloaded each webpage from the data source as an HTML webpage then pulled the charts from that, which is less likely to get you banned than hitting up their site with Selenium.

I then mashed the charts together into one GIANT dataframe which will be helpful for record linkage. This needs properly cleaned as well.

In [8]:
path = 'movie_pages/' #where the HTML files are
listing = os.listdir(path) #read the pages from the directory
all_movie_data = pd.DataFrame(columns = ['Release Date', 'Movie', 'Genre', 'ProductionBudget','DomesticBox Officeto Date', 'Trailer'])

for i, infile in enumerate(listing): #iterate through the files
    temp_list = pd.read_html("movie_pages/"+infile, header=0)
    temp_frame = pd.DataFrame(temp_list[0]) #temp_list is a list of dataframes. Take the first (and only) one
    all_movie_data = all_movie_data.append(temp_frame) #append this to the master dataframe 

In [9]:
del all_movie_data['Trailer'] #not necessary and screwed up the dropna

In [10]:
all_movie_data = all_movie_data.dropna() #drop unneccessary lines with incomplete data

In [11]:
all_movie_data.head()

,Release Date,Movie,Genre,ProductionBudget,DomesticBox Officeto Date
5,"Mar 7, 2008","10,000 B.C.",Adventure,"$105,000,000","$94,784,201"
7,"Mar 11, 2016",10 Cloverfield Lane,Thriller/Suspense,"$5,000,000","$26,790,517"
8,"Nov 11, 2015",10 Days in a Madhouse,Drama,"$12,000,000","$14,616"
28,"Nov 22, 2000",102 Dalmatians,Comedy,"$85,000,000","$66,941,559"
31,"Aug 18, 2006",10th & Wolf,Drama,"$8,000,000","$54,702"


In [12]:
all_movie_data.to_csv("all_movie_data.csv", encoding='utf-8') #used this to prove that record linkage was needed.
# Looked at the data in Excel.

In [33]:
all_movie_data["Movie"] = all_movie_data["Movie"].str.lower()
all_movie_data["ProductionBudget"] = all_movie_data["ProductionBudget"].str.strip(['$'])

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [34]:
all_movie_data

,Release Date,Movie,Genre,ProductionBudget,DomesticBox Officeto Date
5,"Mar 7, 2008","10,000 b.c.",Adventure,NaN,"$94,784,201"
7,"Mar 11, 2016",10 cloverfield lane,Thriller/Suspense,NaN,"$26,790,517"
8,"Nov 11, 2015",10 days in a madhouse,Drama,NaN,"$14,616"
28,"Nov 22, 2000",102 dalmatians,Comedy,NaN,"$66,941,559"
31,"Aug 18, 2006",10th & wolf,Drama,NaN,"$54,702"
35,"Aug 12, 2005",11:14,Drama,NaN,$0
47,"Apr 13, 1957",12 angry men,Drama,NaN,$0
55,"Mar 27, 2009",12 rounds,Action,NaN,"$12,234,694"
60,"Nov 5, 2010",127 hours,Drama,NaN,"$18,335,230"
64,"Apr 23, 2004",13 going on 30,Comedy,NaN,"$57,139,723"


After that, just drop rows with null or empty values because this data is dirty:

### 5. Documentation of the data:

Budget and financial data of movies: http://www.the-numbers.com/movie/budgets/all